# $p$-Value P–P Plot Drawing In-depth Tutorial

# About

This notebook presents how to generate a p-value P–P plot based on subjective data.
Specifically, it shows how to use subjective responses&mdash;expressed on the 5-level Absolute
Category Rating (ACR) scale&mdash;in combination with the software provided in this repository
(cf. [`friendly_gsd.py`](friendly_gsd.py)) to create the p-value P–P plot. The plot can be used
to assess consistency of the subjective data as a whole.

Our recommendation is to use the method across a single experiment only. Differently
put, if you have data from two subjective experiments, please use our software to
generate two separate p-value P–P plots, one for the first and one for the second
experiment.

This document also touches upon various aspects of using the software provided in this repository&mdash;how to
run it, what input does it require, what output it produces and how to use its batch processing functionality.

The content of this notebook is complementary to the following article. Please cite it if you make any use of this repository's content.

```
@inproceedings{Nawała2020,
    author = {Nawa{\l}a, Jakub and Janowski, Lucjan and {\'{C}}miel, Bogdan and Rusek, Krzysztof},
    booktitle = {Proceedings of the 28th ACM International Conference on Multimedia (MM ’20)},
    keywords = {experiment consistency,experiment validity,quality of experience,subjective experiment},
    title = {{Describing Subjective Experiment Consistency by p-Value P–P Plot}},
    year = {2020}
}
```

We stronly encourage you to use the `jupyter_contrib_nbextensions` PIP package in order to display a floating table of contents for this document. This greatly simplifies its consumption. (The relevant nbextension is called "Table of Contents (2)" or "toc2"). 

This notebook was written by Jakub Nawała <jnawala@agh.edu.pl>.

# Overview
Section [High-level Description](#High-level-Description) provides an introductory overview of the method we
use to draw $p$-value P&ndash;P plots.

Section [Running the Software](#Running-the-Software) shows how to run our code. Sections [Output](#Output) and
[Input](#Input) describe what the code outputs and what input information it expects, respectively.

Section [How to Use Different Models](#How-to-Use-Different-Models) higlights what would be necessary in order
to use different score distribution modelling approaches other than the one using the Generalized Score
Distribution (GSD) [Janowski2019].

Section [Batch Processing](#Batch-Processing) presents how to use our code's batch processing functionality.

At last, section [Step-by-step Description](#Step-by-step-Description) brings to light all the details you would
need to reproduce our workflow from the scratch.

# High-level Description

The diagram below (Fig. 1) shows a high-level overview of the methodology. The diagram should be read starting from the top and going downwards. Below the diagram we describe its contents and highlight blocks that require special attention.

![High-level diagram](figures/p-Value_PP_plot_High-level_Explanation.png)
**Figure 1:** The high-level diagram providing an overview of the methodology of creating $p$-value P–P plot.

We start with subjective data (the *Subjective Data* block in the top left corner). These are responses of subjective experiment participants expressed on the 5-level ACR scale. Along with the model describing score distribution of each stimulus (the *Model* block) the responses go into the *MLE* block (the abbreviation stemming from Maximum Likelihood Estimation). This MLE block represents fitting the model to the observed responses. Not surprisingly, this results in the *Fitted Model* block. Importantly, the model is fitted on stimulus-by-stimulus basis. Differently put, if your experiment includes 20 stimuli the method generates 20 fitted models. Each model describes the score distribution of a single stimulus.

Having as many fitted models as there are stimuli in your experiment (the *Fitted Model* block) and the actually observed responses (the *Subjective Data* block) we can proceed to performing the G-test of goodness-of-fit (the *G-test of Goodness-of-Fit* block). It formally checks how well the fitted models describe the observed responses. This step is also performed as many times as there are stimuli in your experiment. Hence, instead of one, this step generates many *p*-values (the *p-Values* block). The resulting *p*-values go into the final *p-Value P–P plot* block. This block represents plotting the target *p*-value P–P plot. This last step depends on the desired significance level (the *Significance Level* block). This level influences the position of the theoretical threshold line (visualised as a solid black line). Importantly, the method yields one P–P plot for one experiment. See Fig. 2 for an exemplary $p$-value P–P plot of an inconsistent subjective experiment.

![Exemplary qq-plot](figures/exemplary_inconsistent_exp_pvalue_pp-plot.png)
**Figure 2:** An exemplary $p$-value P–P plot of an inconsistent experiment.

The blocks coloured blue
correspond to two places where you have to make a decision. The first decision is to choose
the model describing score distribution of a single stimulus (cf. the *Model* block).
We use the Generalised Score Distribution (GSD) here [Janowski2019] (and this is
what is implemented in the [`friendly_gsd.py`](friendly_gsd.py) script), but you are free to choose
any other model (e.g., the model proposed by Li and Bampis [Li2017] or the model proposed by Hossfeld et al. [Hossfeld2020]).
The second decision is to choose the significance level of hypothesis testing related
with drawing the target *p*-value P–P plot (cf. the *Significance Level* block). This significance level defines the position
of the theoretical black line (cf. Eq. (2) in [Nawała2020]). We suggest to keep this
level at 5% (and this value is hard-coded into our [`friendly_gsd.py`](friendly_gsd.py) script; cf. the
*significance_line* variable in the *draw_p_value_qq_plot()* function).

# Running the Software
We show here how to use our software to generate a $p$-value P–P plot (and related G-test results) for exemplary subjective data. We choose to use here the subjective data from the first experiment of the VQEG HDTV Phase I subjective study [Pinson2010].

Assuming the code is run from the root folder of this repository, the following line executed in the terminal
results in a $p$-value P–P plot for the data contained in the `hdtv1_exp1_scores_pp_plot_ready.csv` file:
```
python3 friendly_gsd.py hdtv1_exp1_scores_pp_plot_ready.csv
```

Executing the script this way we ask it to process all 168 stimuli from the first experiment of the HDTV Phase
I test. **Please be adviced that this may take long time to finish**. Our internal trials on consumer-grade laptop (as of
2020) indicate that it takes around 5 minutes to process a single stimulus. This is why the script supports
batch processing. More details on this subject are in [the Batch Processing section](#Batch-Processing).

Next section describes the output produced by the script.

# Output
The `friendly_gsd.py` script produces three types of output
1. P–P plot&mdash;either in the form of a floating figure (default behaviour) or as a file on disk (requires manual modification of the code&mdash;see the text following the list for an explanation what to do).
2. G-test results&mdash;in the form of a CSV file. The filename encodes how many processing chunks were requested (see [the Batch Processing section](#Batch-Processing)) and results of which of these this file contains. When running the code the way [section Running the Software](#Running-the-Software) suggests this filename is as follows: `G-test_chunk_id_0_of_1_chunks.csv`. 
3. Logs&mdash;in the form of a \*.log file. Stores logs of the last run of the script. When running the code the way [section Running the Software](#Running-the-Software) suggests this file takes the following name: `friendly_gsd_chunk_id_0_of_1_chunks.log`.

## Store P&ndash;P Plot on the Disk
To store the resultant P&ndash;P plot to a file, please modify the following line (from the `main()` function of the `friendly_gsd.py` script).
```python
pp_plot_fig_handle = draw_p_value_pp_plot(g_test_res)
```
Add the `should_store_figure=True` argument to the `draw_p_value_pp_plot()` function. After the modification, the
code should look as follows.
```python
pp_plot_fig_handle = draw_p_value_pp_plot(g_test_res, should_store_figure=True)
```

## G-test CSV File Formatting
The CSV file with G-test results has five columns: (i) *stimulus_id*, (ii) *psi_hat*, (iii) *rho_hat*, (iv) *T* and (v)
*p_value*. The first column identifies a stimulus. The second and the third one provide values of the two parameters of
the GSD distribution (that were estimated through MLE based on the sample of scores related with a given
stimulus). The *T* and *p_value* columns express results of the G-test of goodness-of-fit. This test says how well
the GSD distribution with the estimated parameters (*psi_hat* and *rho_hat* columns) fits the score distribution
of a given stimulus&mdash;for more details see [the Step-by-step Description Section](#Step-by-step-Description).

There is a good chance that the only thing you are interested in is the content of *stimulus_id* and *p_value* columns (unless the GSD distribution is more interesting to you than P&ndash;P plots&mdash;in this case take a look at *psi_hat* and *rho_hat* columns).

# Input
The code requires two inputs:
1. A CSV file with subjective scores.
2. The Pickle file with pre-calculated probabilities of each score for a range of psi and rho parameter values. (Psi and rho are the two parameters defining the shape of the GSD distribution.) This file is provided with the code (see the `gsd_prob_grid.pkl` file).

The input CSV file has to comply with the "tidy" data format [Wickham2014]. Only two columns are required: (i)
one identifying a stimulus and (ii) the other one expressing a score assigned to it. By default these columns
should be called *stimulus_id* and *score*, respectively. However, the naming convention can be changed using
*--stimulus-identifier* and *--score-identifier* arguments of the `friendly_gsd.py` script.

Intuitively, if your subjective experiment includes 15 stimuli and 20 participants (each scoring all stimuli) then
your input CSV file should have $15\cdot20=300$ rows (+1 row with the headers). Each stimulus identifier is
repeated 20 times (since 20 participants rate each stimulus).

The [`hdtv1_exp1_scores_pp_plot_ready.csv`](hdtv1_exp1_scores_pp_plot_ready.csv) file can serve as a reference.

Please keep in mind that a CSV file with your subjective data can have more columns. In this situation, the script
is going to use only the two columns described above.

# How to Use Different Models
In case you would like to test our framework using different subjective score distribution models this section
provides some tips on how to do this.

There are few things you will have to take care of.
1. **Provide a pickle file with a probability grid for the model of interest.** To see how this grid should be formatted please refer to the [gsd_prob_grid.pkl](gsd_prob_grid.pkl) file. In short, it should be a two-dimensional Pandas DataFrame with columns identifying a score category and rows identifying a particular value of model parameters. Since the GSD model has two parameters the dimension specifying their values is a MultiIndex. Differently put, each row of the DataFrame is indexed by two numbers: (i) value of the psi parameter and (ii) value of the rho parameter. (This is in contrast to most DataFrames that use single-value indexing on each axis.)
2. **Implement access interface to the model of interest.** It should provide two functionalities: (i) the ability to generate the probability of observing each score category and (ii) the ability to generate a random sample. For an exemplary model access interface please take a look at [gsd.py](gsd.py).
3. **Adapt the `friendly_gsd.perform_g_test()` function**. The current implementation assumes that the model of interest is parametrised by two parameters only. This may not be the case for the model of your choice.

In general, it is not straight forward to adapt our code to work with models significantly different than GSD. We
treat this as an important shortcoming of our implementation and plan to address this problem in any future
implementations.

Importantly, we have not tested our framework with models taking into account individual traits of study
participants (e.g., subject bias [Janowski2015]). (Note that our model can be estimated for a sample of scores assigned to a
single stimulus. To find any individual trait of a study participant one has to analyse this participant's scores
across multiple stimuli. This complicates the analysis.) A model taking into account individual traits can,
however, work with our framework provided its probability grid (see point 1. from the list above) can be
multiplied by a list of observed frequencies of score categories (see the related code fragment from the
`probability_grid_estimation.estimate_parameters()` function) to yield expected probabilities of each score
category.

# Batch Processing
Since performing the bootstrapped version of the G-test of goodness-of-fit (GoF) is computationally-intensive we
designed our code to be batch processing ready. Differently put, you can run multiple instances of the
`friendly_gsd.py` script, each processing a separate part of your input data.

Below is an excerpt from `friendly_gsd.py`'s command-line help message.
```shell
usage: friendly_gsd.py [-h] [-p path] [-c N] [-i idx] [-s identifier]
                       [-o identifier]
                       data_filepath
```
In terms of batch processing the `-c` and `-i` optional arguments are of our interest. Here are their help messages.
```shell
  -c N, --chunks N      (for batch processing) the number of chunks into which
                        the input data should be split. It defaults to 1.
  -i idx, --index idx   (for batch processing) 0-based index of a chunk to
                        process. It default to 0.
```
Using these you can start many instances of `friendly_gsd.py`, each with **the same** input data file, **the same** number
of chunks (`-c`) and each with a **unique** index of a chunk to process (`-i`).

For example, if you have three machines (A, B and C), each with one CPU, it makes sense to start three instances of the
`friendly_gsd.py` script&mdash;one on each of the machines. Now, assuming you have an exact copy of your input
data file (say, `my_input_data.csv`) on each of the machines, here is how you should start the `friendly_gsd.py`
script on each machine.

Machine A
```shell
python3 friendly_gsd.py -c 3 -i 0 my_input_data.csv
```

Machine B
```shell
python3 friendly_gsd.py -c 3 -i 1 my_input_data.csv
```

Machine C
```shell
python3 friendly_gsd.py -c 3 -i 2 my_input_data.csv
```

Please note how the value of the `-i` parameter is changing depending on the machine the script is run on.

After all the computations are finished you end up with one CSV file with G-test results on each machine.
Specifically, on machine A you will get `G-test_chunk_id_0_of_3_chunks.csv`; on machine B:
`G-test_chunk_id_1_of_3_chunks.csv` and on machine C: `G-test_chunk_id_2_of_3_chunks.csv`. These three files
(when combined) contain G-test results for all stimuli present in the `my_input_data.csv` file.

**NOTE**: Make sure the `friendly_gsd.py` script is set up **not** to interactively show *p*-value P&ndash;P plots 
when you use it for batch processing. To suppress interactive plot presentation either comment-out the call
to the `friendly_gsd.draw_p_value_pp_plot` function or ask it to instead store the plots on the disk. The latter
can be achieved by calling the function with the `should_store_figure` keyword argument set to `True`.

# Step-by-step Description
This part of the document looks at subsequent steps necessary to create a *p*-value P–P plot. Use this if you would like to write your own implementation of the pipeline. Apart from describing each step we also give references to related code fragments from our `friendly_gsd.py` script.

## Assumptions
Before we proceed to the steps let us first highlight assumptions we make:
1. Data is processed stimulus-by-stimulus. Differently put, we treat as a single sample of data a set of subjective responses assigned to a single stimulus. For example, the usual data sample from a subjective experiment aiming at assessing image quality is a set of 24 responses (the number coming from an assumption about 24 people participating in the experiment) for each of the tested images (e.g., 300 images). In this case we can say there are 300 data samples, each with 24 observations.
2. Subjective responses are expressed on the 5-level Absolute Category Rating (ACR) scale (cf. Sec. 6.1 of ITU-T Rec. P.910). (Optionally, the responses should be mappable to the 5-level scale. Importantly, we do not provide nor implement such a mapping.)
3. A single data sample is represented as follows: $(n_1, n_2, n_3, n_4, n_5)$, where $n_k$ is the number of responses of category $k$. Significantly, $n = \sum_{k=1}^5 n_k$ and denotes the total number of responses for a given stimulus.
4. The null hypothesis is that the distribution of responses for a single sample of interest follows the assumed model (the GSD model in our case).

## 1. Use MLE to Fit the Model to the Sample
We start with a data sample $(n_1, n_2, n_3, n_4, n_5)$ to which we fit the model.
Having fitted the model, the probability of each response category is as follows: $(p_1, p_2, p_3, p_4, p_5)$, where $p_k$ is the probability of a response of category $k$ (as given by the fitted model).

This functionalitiy is implemented in our `friendly_gsd.py` script in the `friendly_gsd.perform_g_test()` function. Specifically, the following two lines estimate the GSD model parameters ($\psi$ and $\rho$) given the data sample $(n_1, n_2, n_3, n_4, n_5)$ and map these to $(p_1, p_2, p_3, p_4, p_5)$ probabilities.
```python
psi_hat_gsd, rho_hat = estimate_parameters(sample_scores, prob_grid_gsd)
exp_prob_gsd = gsd.prob(psi_hat_gsd, rho_hat)
```

Importantly, to make the computations faster we use a pre-calculated probability grid (cf. the `prob_grid_gsd` variable). It stores probabilities of all response categories for a range of GSD model parameters. For more details please see the "GSD Parameters Estimation" paragraph of [Nawała2020].

## 2. Calculate the T Statistic
The T statistic is the main building block of the G-test of goodness-of-fit (GoF). Since G-test is a likelihood ratio test, the T statistic is defined as a quotient of the likelihood without assumptions about a model describing the sample (this is also called the empirical distribution of the sample) divided by the likelihood when a certain model describing the sample is assumed (the GSD model in our case [Janowski2019]). Hence, the T statistic is calculated as follows:
$$
T = \sum_{k: n_k \neq 0} n_k \ln \left( \frac{n_k}{n p_k} \right)
$$

This functionality is provided in the `friendly_gsd.perform_g_test()` function by the following line.
```python
T_statistic_gsd = bootstrap.T_statistic(score_counts, exp_prob_gsd)
```

## 3. Find the Bootstrap $p$-Value of the T Statistic
Since the total number of responses $n$ for a given stimulus is relatively small in most subjective experiments instead of using the asymptotical distribution of the T statistic we approximate it using bootstrapping. Importantly, we need the distribution of the T statistic in order to calculate the $p$-value of the G-test of GoF.



### 3.1 Generate Bootstrap Samples
Using the probability distribution given by $(p_1, p_2, p_3, p_4, p_5)$ generate $R$ bootstrap samples. In our implementation we use $R=10000$. The higher the $R$ the greater the precision of the $p$-value. Let us denote the $r$-th bootstrap sample as $(m_1, m_2, m_3, m_4, m_5$), where $m_k$ is the number of responses of category $k$. Importantly, each sample should have $n$ responses (the same number as the original, truly observed sample). This last condition can be also formulated as $n = \sum_{k=1}^5 m_k$.

The lines below from the `friendly_gsd.perform_g_test()` function are responsible for generating 10,000 bootstrap samples (the `n_bootstrap_samples` variable has its value set to 10,000).
```python
n_total_scores = np.sum(score_counts)
n_bootstrap_samples = n_bootstrap_samples
bootstrap_samples_gsd = gsd.sample(psi_hat_gsd, rho_hat, n_total_scores, n_bootstrap_samples)
```

### 3.2 Use MLE to fit the Model to Each Bootstrap Sample
Having fitted the model, the probability of each response category is as follows: $(q_1, q_2, q_3, q_4, q_5)$, where $q_k$ is the probability of a response of category $k$ (as given by the fitted model). Please keep in mind that this has to be done for each bootstrap sample (and there are $R$ of these).

The related functionality is provided by the following lines from the `friendly_gsd.perform_g_test()` function.
```python
psi_hat_rho_hat_gsd_bootstrap = np.apply_along_axis(estimate_parameters, axis=1, arr=bootstrap_samples_gsd,
                                                    prob_grid_df=prob_grid_gsd, sample_as_counts=True)

def _get_each_answer_probability(psi_rho_row, prob_generator):
    """
    Translates psi and rho parameters into the probability of each answer.

    :param psi_rho_row: a 2-column vector with the first col. corresponding to psi and the second one to rho
    :param prob_generator: gsd.prob
    :return: a vector of probabilities of each answer
    """
    psi = psi_rho_row[0]
    rho = psi_rho_row[1]
    return prob_generator(psi, rho)

bootstrap_exp_prob_gsd = np.apply_along_axis(_get_each_answer_probability, axis=1,
                                             arr=psi_hat_rho_hat_gsd_bootstrap, prob_generator=gsd.prob)
```

### 3.3 Calculate the T statistic for Each Bootstrap Sample
Proceeding similarly to what was shown in Step 2. we calculate the T statistic for each $r$-th bootstrap sample as follows:
$$
T_r = \sum_{k:m_k \neq 0} m_k \ln \left( \frac{m_k}{n q_k} \right).
$$

Having done that we should have $R$ $T_r$ values (in our case 10,000 $T_r$ values).

This step (and next one) is performed by the following code fragment from the `friendly_gsd.perform_g_test()` function.
```python
p_value_g_test_gsd = bootstrap.G_test(score_counts, exp_prob_gsd, bootstrap_samples_gsd,
                                      bootstrap_exp_prob_gsd)
```

**WARNING**: Our code in the `bootstrap.G_test()` function assumes that the GSD model is used. Thus, if there is only one non-zero response category in $(n_1, n_2, n_3, n_4, n_5)$ or only two neighbouring response categories are non-zero, the code skips the $p$-value computations and sets it to 1.0. (By "non-zero" we mean that there is at least one reponse for a given category.) This behaviour comes from our theoretical analysis of the GSD model. We know for sure that the model completely represents all score distributions with only one or only two neighbouring non-zero response categories. **The takeaway is that you need to modify our code if you are planning on using model different than GSD.** Our recommendation is to simply remove the following lines from the `bootstrap.G_test()` function.
```python
n_non_zero_cells = (n != 0).sum()
if n_non_zero_cells == 1:
    return 1.0

# Return a p-value of 1.0 only if exactly any two NEIGHBOURING cells are non-zero
if n_non_zero_cells == 2:
    # Find indices of the top 2 elements
    top_two_idx = np.argpartition(n, -2)[-2:]
    idx_diff = np.abs(top_two_idx[0] - top_two_idx[1])
    # Only if the top 2 elements are neighbours, return 1.0
    if idx_diff == 1:
        return 1.0
```

### 3.4 Find the Bootstrap $p$-Value
Sort all the $T_r$ values in the ascending order and calculate the bootstrap $p$-value as the number of $T_r$ values that are greater or equal to $T$, divided by the total number $R$ of $T_r$ values. Diffrently put:
$$
p\mbox{-value} = \frac{\# \left( T_r \geq T \right)}{R},
$$
where $\#(\mbox{condition})$ is the number of cases in which the condition is met.

For the related code fragment from our `friendly_gsd.py` script please take a look at Step 3.3.

## 4. Create $p$-Value P–P plot
In order to draw a P–P plot the steps 1. to 3. have to be repeated for each stimuli in the experiment. To be more specific, if your experiment contains 100 stimuli (e.g., images), then steps 1. to 3. have to be repeated 100 times. After all the repetitions, we get a vector of G-test of GoF $p$-values. This vector, along with the assumed significance level, is sufficient to draw the P–P plot.

The very procedure of creating the plot is described by the three following sub-steps.

### 4.1 Check Fraction of Stimuli with $\leq$ $p$-Value
For each $p$-value check what fraction of all stimuli have their $p$-value lower or equal to the $p$-value currently being processed. These fractions constitute $y$-axis values of the red dots visible in Fig. 2. (We also refer to this fraction as $\hat{\alpha}$ in the next step.) Correspondingly, $x$-axis values of the dots are the actual $p$-values of each stimulus (as provided by Step 3.4).

Please note that the way we compute $y$-axis values corresponds to finding empirical cumulative distribution function (ECDF) of the observations. This is why the $y$-axis of the plot in Fig. 2 is labelled as "ecdf of $p$-values."

For an explanation why the $x$-axis is labelled as "theoretical uniform cdf" (although it is used to indicate the actual $p$-values of each stimulus) please refer to Sec. 4 of [Nawała2020].

This functionalitiy is provided by the following lines from the `friendly_gsd.draw_p_value_pp_plot()` function.
```python
def count_pvs_fraction(p_value, p_value_per_pvs):
    return np.sum(p_value_per_pvs <= p_value) / len(p_value_per_pvs)

pvs_fraction_gsd = g_test_res["p_value"].apply(count_pvs_fraction, args=(g_test_res["p_value"],))
```

### 4.2 Draw the Threshold Line
To understand what the threshold line means and where does the significance level come from let us first describe the threshold line drawing procedure for a single $p$-value.

Let us pick $p$-value $= 0.2$. (This is an outcome of Step 3.4 for a singlue stimulus.) Let us label this value as $\alpha=0.2$. Now, we want to know how many stimuli in the whole experiment have their $p$-value below $\alpha$. We take each stimulus and compare its $p$-value with $\alpha$ (cf. Step 4.1 above). If its $p$-value is lower or equal to $\alpha$ we assign the stimulus the value of 1. If, on the other hand, its $p$-value is higher than $\alpha$, we assign it the value of 0. 
We mark the ratio of stimuli assigned 1 to the total number of stimuli as $\hat{\alpha}$.
The meaning of 1 is that a stimulus has low $p$-value with respect to the G-test of goodness-of-fit (GoF). This means the assumed model poorly describes the score distribution of that stimulus (we treat this as a bad outcome). Conversely, the value of 0 means high $p$-value of the G-test of GoF. This corresponds to the assumed model describing the score distribution of the stimulus sufficiently well (we treat this as a good outcome).

Having assigned 1 or 0 to all stimuli we have a vector of 1s and 0s. We can treat this as a random sample from the Bernoulli distribution. Now, if we want to analyse the number of 1s (successes) in such a sample we can observe that this number follows the binomial distribution. This distribution has two parameters conventionally labeled as $p$ and $n$. The first one ($p$) corresponds to the probability of success in each trial. The second one ($n$) defines the number of trials. (If your experiment has 100 stimuli then $n=100$.) Now, we can construct a hypothesis test about $p$ [Siegrist2019]. Staying with $\alpha = 0.2$ we would like to test the following null hypothesis $H_0: p \leq \alpha$ versus the alternative $H_1: p > \alpha$. 

We reject $H_0$ if the following inequality is true:
$$
\begin{align}
\hat{\alpha} > \alpha + z_{1-\beta} \sqrt{\frac{\alpha(1 - \alpha)}{n}}, \label{eq:hyp_test} \tag{1}
\end{align}
$$
where $\beta$ is the significance level of the test and $z_\gamma$ denotes the quantile of order $\gamma$ for the standard normal distribution.

The aforementioned test can either reject or fail to reject $H_0$. How does this then relates to $p$-value P–P plot? If we repeat the procedure for each stimulus in the experiment, each time assuming $\alpha$ equal to stimulus' $p$-value, we get as many hypothesis tests as there are stimuli. To graphically represent Eq. \eqref{eq:hyp_test} we can evaluate its RHS for the complete range of $\alpha$ values (from 0 to 1). We can then plot the result of this evaluation for each value of $\alpha$. This is the threshold line depicted as the solid black line in Fig. 2. Whenever a given stimulus satisfies Eq. \eqref{eq:hyp_test} its related $\hat{\alpha}$ has its value above the treshold line. Simply put, the data point on $p$-value P–P plot related with this stimulus (depicted as red dot) lands above the solid black line.

To link this procedure with practical applications we point out that if many stimuli have data points falling above the threshold line then the experiment in question is most probably inconsistent.

Importantly, even though we mention finding the theoretical line for the range of $\alpha$ values from 0 to 1 (and even though there naturally occur stimuli with their $p$-value above 0.2), the target P–P plot should only depict the $x$-axis range from 0 to 0.2. This is because this range of $\alpha$ values is critical for drawing conclusions. In other words, we are more interested in reasoning related with low $p$-values from the G-test of GoF than we are in making conclusions regarding high $p$-value stimuli.

The following lines from the `friendly_gsd.draw_p_value_pp_plot()` function are responsible for generating the theoretical line.
```python
n_pvs = len(g_test_res)
p_values = pd.Series(np.linspace(start=0.001, stop=thresh_pvalue, num=100))
significance_line = p_values + norm.ppf(0.95) * np.sqrt(p_values * (1 - p_values) / n_pvs)
```

### 4.3 Draw the Plot
Having the $\hat{\alpha}$ values for all the stimuli and the threshold line we can finally draw the $p$-value P–P plot. The $y$-axis corresponds to the $\hat{\alpha}$ values and the $x$-axis to the related $p$-values (from the G-test of GoF). Furthermore, the $y$-axis is also used to express the threshold line (given by the RHS of Eq. (1)). An exemplary $p$-value P–P plot is provided in Fig. 2.

The lines below from the `friendly_gsd.draw_p_value_pp_plot()` function are responsible for drawing the P–P plot.
```python
fig = plt.figure()
plt.scatter(g_test_res["p_value"], pvs_fraction_gsd, label="GSD")
plt.xlabel("theoretical uniform cdf")
plt.ylabel("ecdf of $p$-values")
plt.plot(p_values, significance_line, "-k")
plt.xlim([0, thresh_pvalue])
plt.ylim([0, thresh_pvalue + 0.1])
plt.minorticks_on()
plt.show()
```

# Bibliography
[Janowski2015] Janowski, L., & Pinson, M. (2015). The Accuracy of Subjects in a Quality Experiment: A Theoretical Subject Model. IEEE Transactions on Multimedia, 17(12), 2210–2224. https://doi.org/10.1109/TMM.2015.2484963

[Wickham2014] Wickham, H. (2014). Tidy Data. Journal of Statistical Software, 59(10), 1--23. https://doi.org/10.18637/jss.v059.i10

[Pinson2010] Pinson, M., Speranza, F., Takahashi, A., Schmidmer, C., Lee, C., Okamoto, J., … Dhondt, Y. (2010). Report on the Validation of Video Quality Models for High Definition Video Content (HDTV Phase I). Retrieved from https://www.its.bldrdoc.gov/vqeg/projects/hdtv/hdtv.aspx

[Hossfeld2020] Hossfeld, T., Heegaard, P. E., Varela, M., Skorin-Kapov, L., & Fiedler, M. (2020). From QoS Distributions to QoE Distributions: a System’s Perspective. 4th International Workshop on Quality of Experience Management (QoE Management 2020), Featured by IEEE Conference on Network Softwarization (IEEE NetSoft 2020), Ghent, Belgium, 1–7. Retrieved from http://arxiv.org/abs/2003.12742

[Siegrist2019] Siegrist, K. (2019). Tests in the Bernoulli Model. Retrieved from http://www.randomservices.org/random/hypothesis/Bernoulli.html

[Nawała2020] Nawała, J., Janowski, L., Ćmiel, B., & Rusek, K. (2020). Describing Subjective Experiment Consistency by p-Value P–P Plot. Proceedings of the 28th ACM International Conference on Multimedia (MM ’20).

[Janowski2019] Janowski, L., Ćmiel, B., Rusek, K., Nawała, J., & Li, Z. (2019). Generalized Score Distribution. Retrieved from http://arxiv.org/abs/1909.04369

[Li2017] Li, Z., & Bampis, C. G. (2017). Recover Subjective Quality Scores from Noisy Measurements. Data Compression Conference Proceedings, 52–61. https://doi.org/10.1109/DCC.2017.26